In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

def get_news_info(url, base_url="https://www.investing.com"):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}  # 헤더 설정 추가
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        news_info_list = []

        # 뉴스 제목과 올라온 시간이 포함된 요소 선택
        news_elements = soup.select('.textDiv')

        for element in news_elements:
            title_element = element.select_one('.title')
            date_element = element.select_one('.date')

            if title_element and date_element:
                title_text = title_element.text.strip()
                date_text = date_element.text.strip()

                # 상세 페이지 URL 구성
                news_url = base_url + title_element['href']

                # 상세 페이지로 이동하여 본문 내용을 크롤링
                news_response = requests.get(news_url, headers=headers)
                news_soup = BeautifulSoup(news_response.text, 'html.parser')
                news_content = news_soup.find('div', class_='WYSIWYG articlePage').text.strip()

                news_info_list.append({
                    'title': title_text,
                    'time': date_text,
                    'content': news_content
                })

        return news_info_list

    except Exception as e:
        print(f"에러가 발생했습니다. 페이지: {url}")
        raise e

# 여러 페이지의 뉴스 정보를 저장할 리스트
all_news_info = []

# 각 페이지에 대해 뉴스 정보를 가져와서 리스트에 추가
for page_number in range(3, 4):
    url = f"https://www.investing.com/news/cryptocurrency-news/{page_number}"
    try:
        news_info = get_news_info(url)
        all_news_info.extend(news_info)
        
        # 너무 빠른 요청으로 인한 서버 부하를 방지하기 위해 대기
        time.sleep(1)

        print(f"페이지 {page_number} 크롤링 완료")
    except Exception as e:
        print(f"에러가 발생하여 프로그램을 종료합니다. 페이지: {page_number}")
        break

# 뉴스 정보를 JSON 파일로 저장
result_data = {'news_info': all_news_info}
with open('all_cryptocurrency_news_info.json', 'w', encoding='utf-8') as json_file:
    json.dump(result_data, json_file, ensure_ascii=False, indent=4)

print(f"\n총 {len(all_news_info)}개의 뉴스 정보 크롤링 완료 ")
